In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime

In [44]:
# para comprobar que el codigo funciona correctamente
# usaremos el archivo mas actualizado, los resultados
# deberian ser vacíos
df = pd.read_excel('Tecnoempleo_f.xlsx')

In [35]:
df

,Urls,Oferta,Empresa,Funcion,Aptitudes,Requisito,Idiomas,Jornada,Ubicacion,Teletrabajo o Presencial,Contrato,Salario,Experiencia,Texto,Fecha_de_publicacion,Fecha_extraccion
0,https://www.tecnoempleo.com/programador-fullst...,Programador FullStack,Dev&del,Analista Programador,"['java', 'javascript', 'spring', 'angular', 'r...",FP2/Grado Superior,Inglés (Alto),Jornada completa,Madrid,Híbrido,Indefinido,NaN,3 años,Descripción de la oferta de empleo ¿Eres una p...,17/04/2025,2025-04-17 13:30:13.871
1,https://www.tecnoempleo.com/analista-programad...,Analista Programador en PHP,Codicert,Programador,"['php7', 'phpunit', 'ajax', 'composer', 'poo',...",FP2/Grado Superior,Inglés (Medio),Jornada completa,Gzira - Malta,Presencial,Temporal,30000 30000,2 años,Descripción de la oferta de empleo ¡Únete a nu...,17/04/2025,2025-04-17 13:30:15.038
2,https://www.tecnoempleo.com/senior-backend-dev...,Senior Backend Developer (.NET/ Go/ PHP),WebBeds,Programador,"['C#', 'Golang', 'PHP', 'git', 'kubernetes', '...",Sin estudios,Inglés (Medio),Jornada completa,Palma - Baleares,Híbrido,Indefinido,NaN,3 años,Descripción de la oferta de empleo How will yo...,17/04/2025,2025-04-17 13:30:16.207
3,https://www.tecnoempleo.com/software-engineeri...,Software Engineering Manager,Solera,Analista Programador,"['Java', 'REST', 'API', 'Git', 'CI/CD', 'Hiber...",Ingeniero Superior,Inglés (Alto),Jornada completa,Alcobendas - Madrid - España,NaN,Indefinido,NaN,Más de 5 años,Descripción de la oferta de empleo 7+ years of...,17/04/2025,2025-04-17 13:30:17.374
4,https://www.tecnoempleo.com/front-developer-se...,Front Developer Senior,Mytic Innova,Analista Programador,"['Java', 'Angular', 'SQL', 'SCRUM', 'Git', 'Ht...",Ingeniero Superior,Inglés (Alto),Jornada completa,Hospitalet de Llobregat - Barcelona,Híbrido,Indefinido,27000 33000,2 años,Descripción de la oferta de empleo Mytic Innov...,17/04/2025,2025-04-17 13:30:18.542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,https://www.tecnoempleo.com/administrador-sist...,Administrador Sistemas Linux,Stratio,Administrador,"['linux', 'docker', 'ansible', 'python']",FP2/Grado Superior,NaN,Intensiva Mañana,Pozuelo De Alarcón - Madrid España,NaN,Indefinido,33000 42000,3 años,Descripción de la oferta de empleo En Stratio ...,16/05/2018,2025-04-17 14:47:33.451
3953,https://www.tecnoempleo.com/desarrolladores-ja...,Desarrolladores Java,Stratio,Analista Programador,"['java', 'spring', 'boot', 'Spring', 'Cloud']",Ingeniero Técnico,NaN,Jornada completa,Madrid - España,NaN,Indefinido,NaN,3 años,Descripción de la oferta de empleo Desde Strat...,08/11/2017,2025-04-17 14:47:34.619
3954,https://www.tecnoempleo.com/arquitecto-fullsta...,Arquitecto Fullstack,Stratio,Arquitecto TIC,"['jee', 'java', 'oracle', 'weblogic', 'eclipse']",Ingeniero Superior,NaN,Jornada completa,Madrid - España,NaN,Indefinido,30000 36000,Más de 5 años,Descripción de la oferta de empleo Experiencia...,25/10/2017,2025-04-17 14:47:35.780
3955,https://www.tecnoempleo.com/spark-engineer-mad...,Spark Engineer,Stratio,Arquitecto TIC,"['Spark', 'Hadoop', 'Cassandra', 'Mongo', 'DB'...",Ingeniero Técnico,Español,Jornada completa,Pozuelo - Madrid - España,NaN,Indefinido,NaN,3 años,Descripción de la oferta de empleo Más de 3 añ...,03/10/2017,2025-04-17 14:47:36.950


In [36]:
#FUNCION PARA GENERAR LOS URLS NUEVOS
#ESTA FUNCION HA DE RECIBIR LA ENTRADA DE UN DF DE URLS
def nuevas_ofertas (df_urls):
    
    reps = int(BeautifulSoup(requests.get('https://www.tecnoempleo.com/ofertas-trabajo/?pagina=1').text, "html.parser").find("h1").text.split()[0].replace(".",""))
    links = [f"https://www.tecnoempleo.com/ofertas-trabajo/?pagina={i+1}" for i in range(reps//30+1)]
    
    nuevo = []
    urls = []
    oferta = []
    empresa = []
    
    for i in links:
        response = requests.get(i)
        soup = BeautifulSoup(response.text, "html.parser")
        for j in soup.find_all("div", class_ = "row fs--15"):
            url = j.find("a")["href"]
            if url not in df_urls.values:
                nuevo.append(url)
                try:
                    oferta.append(j.find("h3").find("a")["title"])
                except:
                    oferta.append(np.nan)
                try:
                    empresa.append(j.find("a", class_ = "text-primary link-muted").text)
                except:
                    empresa.append(np.nan)
                
            
    return pd.DataFrame({"Urls": nuevo, "Oferta": oferta, "Empresa": empresa})
    
df_nuevo = nuevas_ofertas(df["Urls"])

In [37]:
df_nuevo

,Urls,Oferta,Empresa


In [38]:
def extracción_nurls (df_urls):
    
    fecha_de_publicacion = []
    fecha_extraccion = []
    aptitudes = []
    texto = []
    funcion = []
    requisito = []
    salario = []
    contrato = []
    jornada = []
    idiomas = []
    ubicacion = []
    experiencia = []
    
    for i in df_urls:
        
        response = requests.get(i)
        soup = BeautifulSoup(response.text, "html.parser")
        numeros = []
        formacion = False
        contra = False
        jorn = False
        idiom = False
        puesto = False
        ubi = False
        text = False
        exp = False
        # EXTRACCION DE DATOS
        # Fecha de oferta
        try:
            fecha_de_publicacion.append(soup.find("span", class_ = "ml-4").text.strip()[:10])
        except:
            fecha_de_publicacion.append(np.nan)

        # Momento de extracción
        fecha_extraccion.append(datetime.now())
    
        # Aptitudes
        try:
            aptitudes.append(soup.find("div", class_= "pl--12 pr--12").text.split())
        except:
            aptitudes.append(np.nan)
            
        # Descripción de la oferta
        for k in soup.find_all("div", class_ = "mt-4"):
            if k.find("p", class_ = "fs--16 text-gray-800"):
                texto.append(" ".join(k.text.split()).strip())
                text = True
                break
        if not text:
            texto.append(np.nan)
    
        # Funcion o puesto de trabajo
        for k in soup.find_all("li", class_ = "list-item clearfix border-bottom py-2"):
            if k.find("i", class_ = "fi fi-users"):
                funcion.append(" ".join([" ".join(i.replace("Funciones", "").split()) for i in (k.text.split())]).strip())
                puesto = True
                break
        if not puesto:
            funcion.append(np.nan)
            
        # Requisitos min
        for k in soup.find_all("div", class_ = "d-flex py-2"):
            if k.find("i", class_ = "fi fi-graduate"):
                requisito.append((k.text.split(":")[1]).strip())
                formacion = True
                break
        if not formacion:
            requisito.append(np.nan)
    
        # Salario min max
        for l in soup.find_all("li", class_ = "list-item clearfix border-bottom py-2"):
            if l.find("i", class_="fi fi-plus"):
                for k in l.text.strip().replace(".","")[:-1].split():
                    if k.isdigit():
                        numeros.append(k)
        salario.append(" ".join(numeros) if numeros else np.nan)
    
        # Tipo de contrato
        for l in soup.find_all("li", class_ = "list-item clearfix border-bottom py-2"):
            if l.find("i", class_ = "fi fi-task-list"):
                contrato.append(" ".join([" ".join(i.replace("Tipo", "").replace("contrato", "").split()) for i in (l.text.split())]).strip())
                contra = True
                break
        if not contra:
            contrato.append(np.nan)
    
        # Jornada (completa, media,etc)
        for l in soup.find_all("li", class_ = "list-item clearfix border-bottom py-2"):
            if l.find("i", class_ = "fi fi-calendar"):
                jornada.append(" ".join(l.text.split()[:2]).strip())
                jorn = True
                break
        if not jorn:
            jornada.append(np.nan)
            
        # Idiomas requeridos
        for l in soup.find_all("div", class_ = "d-flex py-2"):
            if l.find("i", class_ = "fi fi-globe"):
                idiomas.append(" ".join((l.text.replace("|", "").split(":")[1]).split()))
                idiom = True
                break
        if not idiom:
            idiomas.append(np.nan)
    
        # Ubi job
        for k in soup.find_all("li", class_ = "list-item clearfix border-bottom py-2"):
            if k.find("i", class_ = "fi fi-pin"):
                ubicacion.append(" ".join(k.text.split()).strip())
                ubi = True
                break
        if not ubi:
            ubicacion.append(np.nan)
            
        # Experiencia (años)
        for k in soup.find_all("li", class_ = "list-item clearfix border-bottom py-2"):
            if k.find("i", class_ = "fi fi-shield-ok"):
                experiencia.append(" ".join(k.text.split()).replace("Experiencia", "").strip())
                exp = True
                break
        if not exp:
            experiencia.append(np.nan)
        
    sleep(1)

    return pd.DataFrame({"Funcion": funcion,"Aptitudes": aptitudes,
                         "Requisito": requisito,"Jornada": jornada,
                         "Contrato": contrato,"Salario": salario,
                         "Fecha_de_publicacion": fecha_de_publicacion,"Ubicacion": ubicacion,
                         "Idiomas": idiomas,"Texto": texto,
                         "Experiencia": experiencia,"Fecha_extraccion": fecha_extraccion})

df_nuevo1 = extracción_nurls (df_nuevo["Urls"])
df_nuevo1

,Funcion,Aptitudes,Requisito,Jornada,Contrato,Salario,Fecha_de_publicacion,Ubicacion,Idiomas,Texto,Experiencia,Fecha_extraccion


In [39]:
df_final = pd.concat([df_nuevo, df_nuevo1], axis = 1)
df_final

,Urls,Oferta,Empresa,Funcion,Aptitudes,Requisito,Jornada,Contrato,Salario,Fecha_de_publicacion,Ubicacion,Idiomas,Texto,Experiencia,Fecha_extraccion


In [40]:
## De esta manera crearemos la columna de Teletrabajo o Presencial con la de Ubicacion.
def ext_teletrabajo(df_ubicacion):
    teletrabajo = []
    for i in df_ubicacion.values:
        if not pd.isna(i):
            if "Presencial" in i:
                teletrabajo.append("Presencial")
                
            elif "remoto" in i:
                teletrabajo.append("100% remoto")
                
            elif "(Híbrido)" in i:
                teletrabajo.append("Híbrido")
                
            else:
                teletrabajo.append(np.nan)
        else:
            teletrabajo.append(i)
        
    return teletrabajo

df_final["Teletrabajo o Presencial"] = ext_teletrabajo(df_final["Ubicacion"])
df_final

,Urls,Oferta,Empresa,Funcion,Aptitudes,Requisito,Jornada,Contrato,Salario,Fecha_de_publicacion,Ubicacion,Idiomas,Texto,Experiencia,Fecha_extraccion,Teletrabajo o Presencial


In [41]:
df_final = df_final[df.columns]
df_final

,Urls,Oferta,Empresa,Funcion,Aptitudes,Requisito,Idiomas,Jornada,Ubicacion,Teletrabajo o Presencial,Contrato,Salario,Experiencia,Texto,Fecha_de_publicacion,Fecha_extraccion


In [45]:
df_f = pd.concat([df, df_final], ignore_index = True)
df_f

C:\Users\antho\AppData\Local\Temp\ipykernel_274568\1114342675.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_f = pd.concat([df, df_final], ignore_index = True)


,Urls,Oferta,Empresa,Funcion,Aptitudes,Requisito,Idiomas,Jornada,Ubicacion,Teletrabajo o Presencial,Contrato,Salario,Experiencia,Texto,Fecha_de_publicacion,Fecha_extraccion
0,https://www.tecnoempleo.com/programador-fullst...,Programador FullStack,Dev&del,Analista Programador,"['java', 'javascript', 'spring', 'angular', 'r...",FP2/Grado Superior,Inglés (Alto),Jornada completa,Madrid,Híbrido,Indefinido,NaN,3 años,Descripción de la oferta de empleo ¿Eres una p...,17/04/2025,2025-04-17 13:30:13.871
1,https://www.tecnoempleo.com/analista-programad...,Analista Programador en PHP,Codicert,Programador,"['php7', 'phpunit', 'ajax', 'composer', 'poo',...",FP2/Grado Superior,Inglés (Medio),Jornada completa,Gzira - Malta,Presencial,Temporal,30000 30000,2 años,Descripción de la oferta de empleo ¡Únete a nu...,17/04/2025,2025-04-17 13:30:15.038
2,https://www.tecnoempleo.com/senior-backend-dev...,Senior Backend Developer (.NET/ Go/ PHP),WebBeds,Programador,"['C#', 'Golang', 'PHP', 'git', 'kubernetes', '...",Sin estudios,Inglés (Medio),Jornada completa,Palma - Baleares,Híbrido,Indefinido,NaN,3 años,Descripción de la oferta de empleo How will yo...,17/04/2025,2025-04-17 13:30:16.207
3,https://www.tecnoempleo.com/software-engineeri...,Software Engineering Manager,Solera,Analista Programador,"['Java', 'REST', 'API', 'Git', 'CI/CD', 'Hiber...",Ingeniero Superior,Inglés (Alto),Jornada completa,Alcobendas - Madrid - España,NaN,Indefinido,NaN,Más de 5 años,Descripción de la oferta de empleo 7+ years of...,17/04/2025,2025-04-17 13:30:17.374
4,https://www.tecnoempleo.com/front-developer-se...,Front Developer Senior,Mytic Innova,Analista Programador,"['Java', 'Angular', 'SQL', 'SCRUM', 'Git', 'Ht...",Ingeniero Superior,Inglés (Alto),Jornada completa,Hospitalet de Llobregat - Barcelona,Híbrido,Indefinido,27000 33000,2 años,Descripción de la oferta de empleo Mytic Innov...,17/04/2025,2025-04-17 13:30:18.542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,https://www.tecnoempleo.com/administrador-sist...,Administrador Sistemas Linux,Stratio,Administrador,"['linux', 'docker', 'ansible', 'python']",FP2/Grado Superior,NaN,Intensiva Mañana,Pozuelo De Alarcón - Madrid España,NaN,Indefinido,33000 42000,3 años,Descripción de la oferta de empleo En Stratio ...,16/05/2018,2025-04-17 14:47:33.451
3953,https://www.tecnoempleo.com/desarrolladores-ja...,Desarrolladores Java,Stratio,Analista Programador,"['java', 'spring', 'boot', 'Spring', 'Cloud']",Ingeniero Técnico,NaN,Jornada completa,Madrid - España,NaN,Indefinido,NaN,3 años,Descripción de la oferta de empleo Desde Strat...,08/11/2017,2025-04-17 14:47:34.619
3954,https://www.tecnoempleo.com/arquitecto-fullsta...,Arquitecto Fullstack,Stratio,Arquitecto TIC,"['jee', 'java', 'oracle', 'weblogic', 'eclipse']",Ingeniero Superior,NaN,Jornada completa,Madrid - España,NaN,Indefinido,30000 36000,Más de 5 años,Descripción de la oferta de empleo Experiencia...,25/10/2017,2025-04-17 14:47:35.780
3955,https://www.tecnoempleo.com/spark-engineer-mad...,Spark Engineer,Stratio,Arquitecto TIC,"['Spark', 'Hadoop', 'Cassandra', 'Mongo', 'DB'...",Ingeniero Técnico,Español,Jornada completa,Pozuelo - Madrid - España,NaN,Indefinido,NaN,3 años,Descripción de la oferta de empleo Más de 3 añ...,03/10/2017,2025-04-17 14:47:36.950


In [ ]:
# funciona como se esperaba